In [49]:
import os
import torch
import pandas as pd
import random
from tqdm import tqdm

def load_feat(path):
    return torch.load(path)

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x
    return torch.cat((left, right), dim = 0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1
    if concat_n == 1:
        return x
    
    seq_len, feature_len = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n, feature_len).permute(1, 0, 2)
    mid = (concat_n) // 2
    for r_idx in range(1, mid + 1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)
    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_len)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio = 0.8, train_val_seed = 1337):
    class_num = 41
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}

    if mode != 'test':
        phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError("Invalid 'split' argument for dataset: PhoneDataset!")
    
    usage_list = [id.strip('\n') for id in usage_list]
    print('[Dataset] - # Phone classes: ' + str(class_num) + ', number of utterancees for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
        y = torch.empty(max_len, dtype=torch.long)
    
    idx = 0
    for i, frame in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{frame}.pt'))
        cur_len = feat.size(0)
        feat = concat_feat(feat, concat_nframes)
        X[idx:idx + cur_len] = feat
        if mode != 'test':
            y[idx:idx + cur_len] = torch.LongTensor(label_dict[frame])
        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
        y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
        print(y.shape)
        return X, y
    else:
        return X



In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y = None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None
    
    def __getitem__(self, idx):
        if self.label is None:
            return self.data[idx]
        else:
            return self.data[idx], self.label[idx]
        
    def __len__(self):
        return len(self.data)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BasicalBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicalBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        ) 
    def forward(self, x):
        return self.block(x)
    
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim = 41, hidden_layers = 1, hidden_dim = 256):
        super(Classifier, self).__init__()
        
        self.fc = nn.Sequential(
            BasicalBlock(input_dim, hidden_dim),
            *[BasicalBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            BasicalBlock(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.fc(x)

In [41]:
concat_nframes = 7
train_ratio = 0.8
seed = 0
batch_size = 512
num_epoch = 20
learning_rate = 0.0001
model_path = './model.ckpt'
input_dim = 39 * concat_nframes
hidden_layers = 6
hidden_dim = 256

In [42]:
import gc

train_X, train_y = preprocess_data('train', './libriphone/feat', './libriphone', concat_nframes, train_ratio, seed)
valid_X, valid_y = preprocess_data('val', './libriphone/feat', './libriphone', concat_nframes, train_ratio, seed)

train_set = LibriDataset(train_X, train_y)
valid_set = LibriDataset(valid_X, valid_y)

# del train_X, train_y, valid_X, valid_y
# gc.collect()

train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_set, batch_size = batch_size, shuffle = False)


[Dataset] - # Phone classes: 41, number of utterancees for train: 3428


0it [00:00, ?it/s]C:\Users\Doog\AppData\Local\Temp\ipykernel_25408\3476177765.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path)
3428it [00:03, 1119

[INFO] train set
torch.Size([2121270, 273])
torch.Size([2121270])
[Dataset] - # Phone classes: 41, number of utterancees for val: 858


858it [00:00, 1090.93it/s]


[INFO] val set
torch.Size([522888, 273])
torch.Size([522888])


In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'[DEVICE]: {device}')

[DEVICE]: cuda


In [44]:
import numpy as np

In [45]:
def same_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    np.random.seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [46]:
same_seed(seed)
model = Classifier(input_dim = input_dim, hidden_layers = hidden_layers, hidden_dim = hidden_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [47]:
best_acc = 0.0

for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    valid_acc = 0.0
    valid_loss = 0.0

    model.train()
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        preds = model(features)

        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        _, train_pred = torch.max(preds, 1)
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

    if len(valid_set) > 0:
        model.eval()
        with torch.no_grad():
            for i, batch in enumerate(tqdm(valid_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                preds = model(features)

                loss = criterion(preds, labels)

                _, valid_pred = torch.max(preds, 1)
                valid_acc += (valid_pred.cpu() == labels.cpu()).sum().item()
                valid_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Valid Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), valid_acc/len(valid_set), valid_loss/len(valid_loader)
            ))

            if valid_acc > best_acc:
                best_acc = valid_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(valid_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

if len(valid_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch') 

100%|██████████| 1022/1022 [00:04<00:00, 224.42it/s]


[001/020] Train Acc: 0.369456 Loss: 2.609383 | Valid Acc: 0.533967 loss: 1.886129
saving model with acc 0.534


100%|██████████| 1022/1022 [00:04<00:00, 222.40it/s]


[002/020] Train Acc: 0.440023 Loss: 2.260134 | Valid Acc: 0.566500 loss: 1.687021
saving model with acc 0.566


100%|██████████| 1022/1022 [00:04<00:00, 222.18it/s]


...

[018/020] Train Acc: 0.507742 Loss: 1.984467 | Valid Acc: 0.630204 loss: 1.408531
saving model with acc 0.630


100%|██████████| 1022/1022 [00:04<00:00, 211.61it/s]


[019/020] Train Acc: 0.508663 Loss: 1.979960 | Valid Acc: 0.631464 loss: 1.402508
saving model with acc 0.631


100%|██████████| 1022/1022 [00:04<00:00, 212.82it/s]

[020/020] Train Acc: 0.509753 Loss: 1.977051 | Valid Acc: 0.632330 loss: 1.401185
saving model with acc 0.632


In [50]:
# 载入数据
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # Phone classes: 41, number of utterancees for test: 1078


0it [00:00, ?it/s]C:\Users\Doog\AppData\Local\Temp\ipykernel_25408\3229498859.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path)
1078it [00:00, 1256

[INFO] test set
torch.Size([646268, 273])


In [51]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # 获得概率最高的类的索引
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)

100%|██████████| 1263/1263 [00:03<00:00, 351.25it/s]


In [52]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))